<a href="https://colab.research.google.com/github/TomazFilgueira/UFRN-ML-2025-1-Heart_Disease_Classfication/blob/main/heart_disease_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Read Reduced CSV File